In [ ]:
from challenges.mnist.utilitaires import *
from chapters.S_STATS_moyenne_histogramme.utilitaires import *
plt.rcParams['figure.dpi'] = 300

# Belle image

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

d = d_train[0]

# Plot the grayscale image
plt.figure(figsize=(8, 8))
plt.imshow(d, cmap='gray', vmin=0, vmax=255)

# Add text annotations to each pixel
for i in range(d.shape[0]):
    for j in range(d.shape[1]):
        plt.text(j, i, str(d[i, j]), ha='center', va='center', color='red', fontsize=6)

# Hide the axes
plt.axis('off')

# Show the plot
plt.show()


# Analyse statistique de vos caractéristiques

L'objectif de cette section est de mieux comprendre pourquoi une caractéritique donne une erreur plus faible qu'une autre. Une caractéristique doit prendre des valeurs le plus différentes possibles sur chaque type d'image : par exemple des valeurs proches de 0 pour les 7, et proches de 100 pour les 2. Pour mesurer cela nous allons étudier les séries statistiques des valeurs de la caractéristique des images de 2, et des images de 7.

## Séries des caractéristiques

In [ ]:
def caracteristique_ref(d):
        # 1ere caractéristique de référence
        A = (0,0)
        B = (27, 27)
        k = int(moyenne_zone(d, A, B))
        return k

def caracteristique_bas(d):
        # Moyenne sur la moitié basse de l'image
        A = (14,0)
        B = (27, 27)
        k = int(moyenne_zone(d, A, B))
        return k

def caracteristique_boucle(d):
        # Moyenne sur la boucle des 2 en bas à gauche
        A = (16,4)
        B = (24, 12)
        k = int(moyenne_zone(d, A, B))
        return k

def caracteristique_MG(d):
        # Moyenne sur la moitié gauche de l'image
        A = (0,0)
        B = (27, 13)
        k = int(moyenne_zone(d, A, B))
        return k

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('colheader_justify', 'center')

c_train_ref = [caracteristique_ref(d) for d in d_train]
c_train_ref_2, c_train_ref_7 = np.array(c_train_ref)[r_train==2], np.array(c_train_ref)[r_train==7]
l = min(len(c_train_ref_2), len(c_train_ref_7))
df_ref = pd.DataFrame({'k 2':c_train_ref_2[:l], 'k 7' :c_train_ref_7[:l]})
#df_ref.head()

c_train_bas = [caracteristique_bas(d) for d in d_train]
c_train_bas_2, c_train_bas_7 = np.array(c_train_bas)[r_train==2], np.array(c_train_bas)[r_train==7]
df_bas = pd.DataFrame({'k 2':c_train_bas_2[:l], 'k 7' :c_train_bas_7[:l]})
#df_bas.head()

c_train_boucle = [caracteristique_boucle(d) for d in d_train]
c_train_boucle_2, c_train_boucle_7 = np.array(c_train_boucle)[r_train==2], np.array(c_train_boucle)[r_train==7]
df_boucle = pd.DataFrame({'k 2':c_train_boucle_2[:l], 'k 7' :c_train_boucle_7[:l]})

df_ref.columns = ['Images de 2', 'Images de 7']
df_bas.columns = ['Images de 2', 'Images de 7']
df_boucle.columns = ['Images de 2', 'Images de 7']

In [ ]:
from IPython.display import display_html
from itertools import chain, cycle

def display_side_by_side(*args, titles=cycle([''])):
    html_str = ''
    for df, title in zip(args, chain(titles, cycle(['</br>']))):
        html_str += '<div style="display: inline-block; text-align: center; vertical-align: top; margin-right: 20px;">'
        html_str += f'<h2 style="text-align: center; margin: 0;">{title}</h2>'
        html_str += df.to_html(index=False).replace('table', 'table style="display:inline; margin: 0 auto;"')
        html_str += '</div>'
    display_html(html_str, raw=True)



display_side_by_side(df_ref[:10], df_bas[:10], df_boucle[:10], titles=['Caractéristique de référence', 'Caractéristique basse', 'Caractéristique boucle'])


**Questions** : on donne la série des caractéristiques sur les images de 2 et de 7 pour la caractéristique de référence (10 premières images de chaque classe)
- que vaut la moyenne pour chaque classe ?

De même pour une 2e caractéristique (boucle)
- que vaut la moyenne pour chaque classe ?

- quelle caractériostique donnera l'erreur la plus faible ?

## Frise image et caractéristiques

In [ ]:
# Frise de 10 images de 2 avec leur caractéristiques

caracteristique = caracteristique_ref

images_2 = d_train[r_train == 2]
c_2 = [caracteristique(d) for d in images_2]
c_2_fake = [13, 11, 12, 12, 13, 14, 12, 13, 12, 11]

plt.figure(figsize=(10, 1))
for i in range(10):
    plt.subplot(1, 10, i + 1)
    plt.imshow(images_2[i], cmap='gray', vmin=0, vmax=255)
    plt.title(f'$k = {c_2_fake[i]}$', fontsize=10)
    plt.axis('off')
plt.show()



In [ ]:
# Frise de 10 images de 7 avec leur caractéristiques

caracteristique = caracteristique_ref

images_7 = d_train[r_train == 7]
c_7 = [caracteristique(d) for d in images_7]
c_7_fake = [15, 14, 12, 16, 15, 16, 16, 13, 14, 15]

plt.figure(figsize=(10, 1))
for i in range(10):
    plt.subplot(1, 10, i + 1)
    plt.imshow(images_7[i], cmap='gray', vmin=0, vmax=255)
    plt.title(f'$k = {c_7_fake[i]}$', fontsize=10)
    plt.axis('off')
plt.show()


## Histogramme de la caractéristique

Plus la caractéristiquer prend des valeur différentes sur les images de 2 et de 7, moins on fera d'erreur. Il faut donc comparer deux séries statistiques : 
- la série des valeurs de la caractéristique sur les images de 7
- la série des valeurs de la caractéristique sur les images de 7

Pour visualiser la répartition des valeurs de chaque série, on trace leur histogramme (ou diagramme en barre). Chaque 

In [ ]:
visualiser_histogrammes_mnist_2(c_train_ref, size='grand', thick_ticks=True)
visualiser_histogrammes_mnist_2(c_train_bas, size='grand', thick_ticks=True)

In [ ]:
# Changer les valeurs des coordonnées de A et de B avec des valeurs entre 0 et 27 (numéro de ligne puis colonne)
A = (0, 0)       # <- essayer d'autres coordonnées !
B = (27, 27)     # <- essayer d'autres coordonnées !


# La fonction affichage peut prendre en paramètre les coordonnées de la zone choisie pour l'entourer en rouge
def caracteristique(d):
        # La fonction moyenne_zone cacule la moyenne des pixels entre les points A et B
        k = moyenne_zone(d, A, B)
        return k

c_train = []

for image in d_train:
    c_train.append(caracteristique(image))

affichage_dix(d_train, A, B)
#affichage_2_simple(d, d_train[2], A, B, True)

visualiser_histogrammes_mnist_2(c_train, size='grand', thick_ticks=True)

In [ ]:
from matplotlib.ticker import AutoMinorLocator, MultipleLocator

### VERSION 2 7

# Visualiser les histogrammes
def visualiser_histogrammes_mnist_2(c_train, size='grand', legend_loc='upper right', x_min = None, x_max = None, y_max = None, legende = False, legende_y = '\n Nombre d\'images\nde caractéristique $k$', thick_ticks=False, r_train=r_train):
    if size == 'grand':
        font_size = 14
    elif size == 'petit':
        font_size = 17
    else:
        raise ValueError("size doit valoir 'grand' ou 'petit'") 
    
    pas = 25
    mini_pas = 5
    if x_min is None:
        x_min = int(min(c_train)/pas)*pas
    
    if x_max is None:
        x_max = int(max(c_train)/pas)*pas+pas
    
    nb_digits = len(chiffres)
    c_train_par_population = [np.array(c_train)[r_train==k] for k in classes]

    # Deux premières couleurs par défaut de Matplotlib
    colors = ['C0', 'C1']

    fig, ax = plt.subplots(figsize=(6,4))
    # Visualisation des histogrammes
    labels_hist = ['$h$', '$g$']

    # Création des bins :
    bins = np.arange(x_min, x_max+1, mini_pas)

    max_histo = []
    for k in range(nb_digits):
        #label = "$y = $"+str(classes[k])+" (chiffre : "+str(chiffres[k])+")"
        #label = labels_hist[k]+" ($y=$"+str(classes[k])+", chiffre : "+str(chiffres[k])+")"
        if legende:
            label = labels_hist[k]+" ($y=$"+str(chiffres[k])+")"
        else:
            #label = "$y=$"+str(chiffres[k])
            label = str(chiffres[k])
        histo = ax.hist(c_train_par_population[k], bins=bins, alpha=0.7, density=False, 
            label=label, edgecolor='black')
        max_histo.append(max(histo[0]))
    
    if y_max is None:
        y_max = max(max_histo)+0.1*max(max_histo)

    ax.set_xlim(xmin=x_min)
    ax.set_xlim(xmax=x_max)
    ax.set_ylim(ymax = y_max)
    #ax.set_title("Histogrammes de la caractéristique")
    ax.legend(loc=legend_loc, fontsize=font_size+2)
    
    # Ticks principaux et secondaires sur x
    ax.xaxis.set_major_locator(MultipleLocator(pas))
    ax.xaxis.set_minor_locator(MultipleLocator(mini_pas))

    # Ticks principaux et secondaires sur y
    ecart = int((y_max//10)/100)*100+100
    #ecart = 100
    ax.yaxis.set_major_locator(MultipleLocator(ecart))
    ax.yaxis.set_minor_locator(MultipleLocator(ecart//5))

    # Font size pour les ticks
    ax.tick_params(axis='both', labelsize=font_size)
    # Ticks épais si demandé:
    if thick_ticks:
        ax.tick_params(which='major', length=8, color='black', width=2.5)
        ax.tick_params(which='minor', length=5, color='black', width=1.5)

    # Enlever les axes de droites et du haut
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    
    # Centrer les axes en (0,0)
    ax.spines['left'].set_position(('data', x_min))
    ax.spines['bottom'].set_position(("data", 0))

    
    #Afficher les flèches au bout des axes
    ax.plot(1, 0, ">k", transform=ax.get_yaxis_transform(), clip_on=False)
    ax.plot(x_min, 1, "^k", transform=ax.get_xaxis_transform(), clip_on=False)   
    
    # Nom des axex
    ax.set_xlabel('$k$', loc='right', fontsize=font_size)
    if legende:
        ax.set_ylabel('$h$ et $g$', loc='top', rotation='horizontal', fontsize=font_size)
    else:
        ax.set_ylabel(legende_y, rotation='horizontal', horizontalalignment='left', fontsize=font_size-4, labelpad=-180)
        ax.yaxis.set_label_coords(0.025, 0.95)

    plt.show()
    plt.close()
    print(x_min, x_max, y_max)


In [ ]:
from matplotlib.ticker import AutoMinorLocator, MultipleLocator

### VERSION 2 7

# Visualiser les histogrammes
def visualiser_histogrammes_frise(c_train_liste, figsize_single = (6,4), shape=None, size='grand', legend_loc='upper right', x_min = None, x_max = None, y_max = None, legende = False, legende_y = '\n Nombre d\'images\nde caractéristique $k$', thick_ticks=False):

    if size == 'grand':
        font_size = 14
    elif size == 'petit':
        font_size = 17
    else:
        raise ValueError("size doit valoir 'grand' ou 'petit'") 
    
    n = len(c_train_liste)
    if shape is None:
        L, C = 1, n
    else:
        if shape[0]*shape[1] >= n:
            L, C = shape[0], shape[1]
        else:
            raise ValueError("shape must a tuple (L, C) with L*C>=len(c_train_list)") 

    pas = 25
    mini_pas = 5

    mins_ = [min(c_train) for c_train in c_train_liste]
    maxs_ = [max(c_train) for c_train in c_train_liste]

    if x_min is None:
        x_min = int(min(mins_)/pas)*pas
    
    if x_max is None:
        x_max = int(max(maxs_)/pas)*pas
    
    nb_digits = len(chiffres)
    c_train_par_population_liste = [[np.array(c_train)[r_train==k] for k in classes] for c_train in c_train_liste]

    # Deux premières couleurs par défaut de Matplotlib
    colors = ['C0', 'C1']

    figsize = (figsize_single[0]*C, figsize_single[1]*L)
    fig, ax = plt.subplots(L, C, figsize=figsize)
    ax.shape = (L,C)

    # Visualisation des histogrammes
    labels_hist = ['$h$', '$g$']

    # Création des bins :
    bins = np.arange(x_min, x_max+1, mini_pas)

    max_histo = []
    count = 0
    for i in range(L):
        for j in range(C):
            if count < n:
                c_train_par_population = c_train_par_population_liste[count]
                for k in range(nb_digits):
                    #label = "$y = $"+str(classes[k])+" (chiffre : "+str(chiffres[k])+")"
                    #label = labels_hist[k]+" ($y=$"+str(classes[k])+", chiffre : "+str(chiffres[k])+")"
                    if legende:
                        label = labels_hist[k]+" ($y=$"+str(chiffres[k])+")"
                    else:
                        #label = "$y=$"+str(chiffres[k])
                        label = str(chiffres[k])
                    histo = ax[i,j].hist(c_train_par_population[k], bins=bins, alpha=0.7, density=False, 
                        label=label, edgecolor='black')
                    
                ax[i,j].set_xlim(xmin=x_min)
                ax[i,j].set_xlim(xmax=x_max)
                #ax[i,j].set_ylim(ymax = y_max)
                #ax.set_title("Histogrammes de la caractéristique")
                ax[i,j].legend(loc=legend_loc, fontsize=font_size+2)
                
                # Ticks principaux et secondaires sur x
                ax[i,j].xaxis.set_major_locator(MultipleLocator(pas))
                ax[i,j].xaxis.set_minor_locator(MultipleLocator(mini_pas))

                # Ticks principaux et secondaires sur y
                #ecart = int((y_max//10)/100)*100+100
                #ecart = 100
                #ax[i,j].yaxis.set_major_locator(MultipleLocator(ecart))
                #ax[i,j].yaxis.set_minor_locator(MultipleLocator(ecart//5))

                # Font size pour les ticks
                ax[i,j].tick_params(axis='both', labelsize=font_size)
                # Ticks épais si demandé:
                if thick_ticks:
                    ax[i,j].tick_params(which='major', length=8, color='black', width=2.5)
                    ax[i,j].tick_params(which='minor', length=5, color='black', width=1.5)

                # Enlever les axes de droites et du haut
                ax[i,j].spines['right'].set_visible(False)
                ax[i,j].spines['top'].set_visible(False)
                
                # Centrer les axes en (0,0)
                ax[i,j].spines['left'].set_position(('data', x_min))
                ax[i,j].spines['bottom'].set_position(("data", 0))

                
                #Afficher les flèches au bout des axes
                ax[i,j].plot(1, 0, ">k", transform=ax[i,j].get_yaxis_transform(), clip_on=False)
                ax[i,j].plot(x_min, 1, "^k", transform=ax[i,j].get_xaxis_transform(), clip_on=False)   
                
                # Nom des axex
                ax[i,j].set_xlabel('$k$', loc='right', fontsize=font_size)
                if legende:
                    ax[i,j].set_ylabel('$h$ et $g$', loc='top', rotation='horizontal', fontsize=font_size)
                else:
                    ax[i,j].set_ylabel(legende_y, rotation='horizontal', horizontalalignment='left', fontsize=font_size-4, labelpad=-180)
                    ax[i,j].yaxis.set_label_coords(0.025, 0.95)

                max_histo.append(max(histo[0]))
                count +=1
            else:
                ax[i,j].axis('off')
    
    if y_max is None:
        y_max = max(max_histo)+0.05*max(max_histo)

    
    plt.show()
    plt.close()
    #print(n, L, C)


In [ ]:
c_train_liste = [c_train_boucle, c_train_bas, c_train_ref]
visualiser_histogrammes_frise(c_train_liste, shape=(2,2))

In [ ]:

def affichage_2_simple(image1, image2, A=None, B=None, displayPoints=False, titre1="", titre2=""):
    """Fonction qui affiche deux images côte à côté, avec un rectangle rouge délimité par les points A et B
        A : tuple (ligne, colonne) représentant le coin en haut à gauche du rectangle
        B : tuple (ligne, colonne) représentant le coin en bas à droite du rectangle
    Si displayPoints est True, les points A et B sont affichés
    """
    if image1.min().min() < 0 or image1.max().max() > 255:
        print_error("fonction affichage : Les valeurs des pixels des images doivent être compris entre 0 et 255.")
        return
    
    if image2.min().min() < 0 or image2.max().max() > 255:
        print_error("fonction affichage : Les valeurs des pixels des images doivent être compris entre 0 et 255.")
        return

    fig, ax = plt.subplots(1, 2, figsize=(5,2))
    imshow(ax[0], image1)
    ax[0].set_title(titre1)
    ax[0].set_xticks(np.arange(0,28,5))
    ax[0].xaxis.tick_top()
    outline_selected(ax[0], A, B, displayPoints)

    imshow(ax[1], image2)
    ax[1].set_title(titre2)
    ax[1].set_xticks(np.arange(0,28,5))
    ax[1].xaxis.tick_top()
    outline_selected(ax[1], A, B, displayPoints)

    plt.show()
    plt.close()
    return


In [ ]:
c_train = []

for image in d_train:
    c_train.append(caracteristique(image))

visualiser_histogrammes_mnist_2(c_train, size='petit', thick_ticks=True)

In [ ]:
max(c_train), min(c_train)

In [ ]:
np.arange(10, 137, 4)

# Test Chart js

In [ ]:
import iplotter
from IPython.core.display import HTML

In [ ]:
# remove iFrame border for cleaner chart rendering
# increase size of text explanations
HTML("""
<style>
iframe {border:0;}
</style>
""")

In [ ]:
# define chart + data
chart = {
    "data": {
        "columns": [
            ["setosa_x", 3.5, 3.0, 3.2, 3.1, 3.6, 3.9, 3.4, 3.4, 2.9, 3.1, 3.7, 3.4, 3.0, 3.0, 4.0, 4.4, 3.9, 3.5, 3.8, 3.8, 3.4, 3.7, 3.6, 3.3, 3.4, 3.0, 3.4, 3.5, 3.4, 3.2, 3.1, 3.4, 4.1, 4.2, 3.1, 3.2, 3.5, 3.6, 3.0, 3.4, 3.5, 2.3, 3.2, 3.5, 3.8, 3.0, 3.8, 3.2, 3.7, 3.3],
            ["versicolor_x", 3.2, 3.2, 3.1, 2.3, 2.8, 2.8, 3.3, 2.4, 2.9, 2.7, 2.0, 3.0, 2.2, 2.9, 2.9, 3.1, 3.0, 2.7, 2.2, 2.5, 3.2, 2.8, 2.5, 2.8, 2.9, 3.0, 2.8, 3.0, 2.9, 2.6, 2.4, 2.4, 2.7, 2.7, 3.0, 3.4, 3.1, 2.3, 3.0, 2.5, 2.6, 3.0, 2.6, 2.3, 2.7, 3.0, 2.9, 2.9, 2.5, 2.8],
            ["setosa", 0.2, 0.2, 0.2, 0.2, 0.2, 0.4, 0.3, 0.2, 0.2, 0.1, 0.2, 0.2, 0.1, 0.1, 0.2, 0.4, 0.4, 0.3, 0.3, 0.3, 0.2, 0.4, 0.2, 0.5, 0.2, 0.2, 0.4, 0.2, 0.2, 0.2, 0.2, 0.4, 0.1, 0.2, 0.2, 0.2, 0.2, 0.1, 0.2, 0.2, 0.3, 0.3, 0.2, 0.6, 0.4, 0.3, 0.2, 0.2, 0.2, 0.2],
            ["versicolor", 1.4, 1.5, 1.5, 1.3, 1.5, 1.3, 1.6, 1.0, 1.3, 1.4, 1.0, 1.5, 1.0, 1.4, 1.3, 1.4, 1.5, 1.0, 1.5, 1.1, 1.8, 1.3, 1.5, 1.2, 1.3, 1.4, 1.4, 1.7, 1.5, 1.0, 1.1, 1.0, 1.2, 1.6, 1.5, 1.6, 1.5, 1.3, 1.3, 1.3, 1.2, 1.4, 1.2, 1.0, 1.3, 1.2, 1.3, 1.3, 1.1, 1.3]
        ],
     "type": 'scatter'
            },
    "axis": {
        "x": {
            "label": 'Sepal.Width',
            "tick": {
                "fit": "false"
            }
        },
        "y": {
            "label": 'Petal.Width'
        }
    }
}

In [ ]:
c3_plotter = iplotter.C3Plotter()
c3_plotter.plot(chart)

In [ ]:
labels = ["Red", "Blue", "Yellow", "Green", "Purple", "Orange"]

In [ ]:
data = {    
    "labels": labels,
    "datasets": [
        {
            "label": "My First dataset",
            "backgroundColor": "rgba(179,181,198,0.2)",
            "borderColor": "rgba(179,181,198,1)",
            "pointBackgroundColor": "rgba(179,181,198,1)",
            "pointBorderColor": "#fff",
            "pointHoverBackgroundColor": "#fff",
            "pointHoverBorderColor": "rgba(179,181,198,1)",
            "data": [65, 59, 90, 81, 56, 55, 40]
        }
    ]
}

In [ ]:
chart_js = iplotter.ChartJSPlotter()

chart_js.plot(data, chart_type="bar", w=600, h=600)

In [ ]:
import numpy as np
from ipywidgets import HBox, VBox
#from chartjs import ChartJS, BarChart
from traitlets import Unicode, Instance, Bool, Int, List, Dict

# Your function to generate data
def generate_data(size):
    return np.random.normal(size=size)

data = generate_data(100)

# Compute the bins for the histogram and the frequency of each bin
bins = np.linspace(min(data), max(data), 15)
freq, _ = np.histogram(data, bins=bins)

# Create an instance of the ChartJSPlotter
chart_js = ChartJSPlotter()

# Set properties for the chart
chart_js.x_title = 'Value'
chart_js.y_title = 'Frequency'
chart_js.title = 'Histogram of Values'
chart_js.x_labels = list(map(lambda x: str(round(x, 2)), bins[:-1]))

# Plot the chart
chart_js.plot(freq, chart_type="bar", w=600, h=600)


In [ ]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from traitlets import link
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [ ]:
a_slider = widgets.FloatSlider(value=0.0, min=-10.0, max=10.0, step=0.1, description='a:')
b_slider = widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='b:')

In [ ]:
def generate_data(a, b):
    # Replace this with your custom data generation logic
    return np.random.normal(loc=a, scale=b, size=100)

def update_histogram(change):
    a = a_slider.value
    b = b_slider.value
    data = generate_data(a, b)

    # Update the histogram data
    histogram.x = data

    # Update the plot layout with the new data
    fig.data = [histogram]
    fig.update_layout(
        title='Histogram of Values',
        xaxis_title='Value',
        yaxis_title='Frequency',
        width=600,
        height=400,
    )

In [ ]:
# Initialize the chart with some default data
a_init, b_init = a_slider.value, b_slider.value
data = generate_data(a_init, b_init)

# Create a histogram object
histogram = go.Histogram(x=data)

# Create a figure with the histogram
fig = make_subplots(specs=[[{"type": "histogram"}]])
fig.add_trace(histogram)
fig.update_layout(
    title='Histogram of Values',
    xaxis_title='Value',
    yaxis_title='Frequency',
    width=600,
    height=400,
)


In [ ]:
a_slider.observe(update_histogram, names='value')
b_slider.observe(update_histogram, names='value')

In [ ]:
display(widgets.HBox([a_slider, b_slider]))
display(fig)

# Sliders pour la zone et mise à jour des histogrammes

In [ ]:
# Visualiser les histogrammes
def visualiser_histogrammes_zone(c_train, A, B, image, size='grand', legend_loc='upper right', x_min = None, x_max = None, y_max = None, legende = False, legende_y = '\n Nombre d\'images\nde caractéristique $k$', thick_ticks=False):
    if size == 'grand':
        font_size = 14
    elif size == 'petit':
        font_size = 17
    else:
        raise ValueError("size doit valoir 'grand' ou 'petit'") 
    
    pas = 25
    mini_pas = 5
    if x_min is None:
        x_min = int(min(c_train)/pas)*pas
    
    if x_max is None:
        x_max = int(max(c_train)/pas)*pas+pas
    
    nb_digits = len(chiffres)
    c_train_par_population = [np.array(c_train)[r_train==k] for k in classes]

    # Deux premières couleurs par défaut de Matplotlib
    colors = ['C0', 'C1']

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,4))

    imshow(ax1, image)
    ax1.set_xticks(np.arange(0,28,5))
    ax1.xaxis.tick_top()
    #ax1.set_title(titre)
    ax1.xaxis.set_label_position('top') 
    outline_selected(ax1, A, B, False)

    # Visualisation des histogrammes
    labels_hist = ['$h$', '$g$']

    # Création des bins :
    bins = np.arange(x_min, x_max+1, mini_pas)

    max_histo = []
    for k in range(nb_digits):
        #label = "$y = $"+str(classes[k])+" (chiffre : "+str(chiffres[k])+")"
        #label = labels_hist[k]+" ($y=$"+str(classes[k])+", chiffre : "+str(chiffres[k])+")"
        if legende:
            label = labels_hist[k]+" ($y=$"+str(chiffres[k])+")"
        else:
            #label = "$y=$"+str(chiffres[k])
            label = str(chiffres[k])
        histo = ax2.hist(c_train_par_population[k], bins=bins, alpha=0.7, density=False, 
            label=label, edgecolor='black')
        max_histo.append(max(histo[0]))
    
    if y_max is None:
        y_max = max(max_histo)+0.1*max(max_histo)

    ax2.set_xlim(xmin=x_min)
    ax2.set_xlim(xmax=x_max)
    ax2.set_ylim(ymax = y_max)
    #ax.set_title("Histogrammes de la caractéristique")
    ax2.legend(loc=legend_loc, fontsize=font_size+2)
    
    # Ticks principaux et secondaires sur x
    ax2.xaxis.set_major_locator(MultipleLocator(pas))
    ax2.xaxis.set_minor_locator(MultipleLocator(mini_pas))

    # Ticks principaux et secondaires sur y
    ecart = int((y_max//10)/100)*100+100
    #ecart = 100
    ax2.yaxis.set_major_locator(MultipleLocator(ecart))
    ax2.yaxis.set_minor_locator(MultipleLocator(ecart//5))

    # Font size pour les ticks
    ax2.tick_params(axis='both', labelsize=font_size)
    # Ticks épais si demandé:
    if thick_ticks:
        ax2.tick_params(which='major', length=8, color='black', width=2.5)
        ax2.tick_params(which='minor', length=5, color='black', width=1.5)

    # Enlever les axes de droites et du haut
    ax2.spines['right'].set_visible(False)
    ax2.spines['top'].set_visible(False)
    
    # Centrer les axes en (0,0)
    ax2.spines['left'].set_position(('data', x_min))
    ax2.spines['bottom'].set_position(("data", 0))

    
    #Afficher les flèches au bout des axes
    ax2.plot(1, 0, ">k", transform=ax2.get_yaxis_transform(), clip_on=False)
    ax2.plot(x_min, 1, "^k", transform=ax2.get_xaxis_transform(), clip_on=False)   
    
    # Nom des axex
    ax2.set_xlabel('$k$', loc='right', fontsize=font_size)
    if legende:
        ax2.set_ylabel('$h$ et $g$', loc='top', rotation='horizontal', fontsize=font_size)
    else:
        ax2.set_ylabel(legende_y, rotation='horizontal', horizontalalignment='left', fontsize=font_size-4, labelpad=-180)
        ax2.yaxis.set_label_coords(0.025, 0.95)

    plt.show()
    plt.close()
    print(x_min, x_max, y_max)


In [ ]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
import ipywidgets as widgets
import numpy as np

# Define your data generation function
def generate_data(a, b):
    # This is just an example, replace with your actual function
    return np.random.normal(a, b, 1000)

# Define the function to plot data
def plot_data(a, b):
    A = (7, 7)       # <- essayer d'autres coordonnées !
    B = (a, b)     # <- essayer d'autres coordonnées !

    def caracteristique(d):
        # La fonction moyenne_zone cacule la moyenne des pixels entre les points A et B
        k = moyenne_zone(d, A, B)
        return k
    
    c_train = [caracteristique(d) for d in d_train]
    clear_output(wait=True)
    visualiser_histogrammes_zone(c_train, A, B, d, size='grand', thick_ticks=True)

# Create sliders
a_slider = widgets.IntSlider(min=0, max=27, step=1, value=10)
b_slider = widgets.IntSlider(min=0, max=27, step=1, value=10)

# Create interactive widget
widgets.interact(plot_data, a=a_slider, b=b_slider)

In [ ]:
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
import numpy as np
import json

# Define your data generation function
def generate_data(a, b):
    # This is just an example, replace with your actual function
    return np.random.normal(a, b, 1000).tolist()

# Define the function to plot data
def plot_data(a, b):
    data = generate_data(a, b)
    clear_output(wait=True)
    display(HTML("""
        <div style="width: 100%;"><canvas id="myChart"></canvas></div>
        <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
        <script>
        var ctx = document.getElementById('myChart').getContext('2d');
        var myChart = new Chart(ctx, {
            type: 'bar',
            data: {
                labels: Array.from({length: 30}, (_, i) => i + 1),
                datasets: [{
                    label: 'Data',
                    data: """ + json.dumps(data) + """,
                    backgroundColor: 'rgba(75, 192, 192, 0.2)',
                    borderColor: 'rgba(75, 192, 192, 1)',
                    borderWidth: 1
                }]
            },
            options: {
                scales: {
                    y: {
                        beginAtZero: true
                    }
                }
            }
        });
        </script>
    """))

# Create sliders
a_slider = widgets.IntSlider(min=0, max=10, step=1, value=5)
b_slider = widgets.IntSlider(min=0, max=10, step=1, value=5)

# Create interactive widget
widgets.interact(plot_data, a=a_slider, b=b_slider)

# Animation histogramme

In [ ]:
c_train = np.array(c_train)

# Les valeurs entre a et b
a = 20
b = 50

c_train_2 = c_train[r_train == 2]
c_train_7 = c_train[r_train == 7]

c_train_a_b_2 = c_train_2[(c_train_2 >= a) & (c_train_2 <= b)]
c_train_a_b_7 = c_train_7[(c_train_7 >= a) & (c_train_7 <= b)]

r_train_20_50 = r_train[(c_train >= a) & (c_train <= b)]

# print sizes
print("Taille de r_train_20_50 :", len(r_train_20_50))
print("Taille de c_train_20_50_2 :", len(c_train_a_b_2))
print("Taille de c_train_20_50_7 :", len(c_train_a_b_7))


In [ ]:
# histogramme des valeurs entre a et b, bins de taille 2, x_min = a, x_max = b
plt.hist(c_train_a_b_2, bins=np.arange(a, b+2, 2), edgecolor='black', label='Images de 2')
plt.hist(c_train_a_b_7, bins=np.arange(a, b+2, 2), edgecolor='black', alpha=0.5, label='Images de 7')
plt.xticks(np.arange(a, b+2, 2))
plt.legend()
plt.show()

In [ ]:
# cheating for a better visualization
# removing the 2 with c_train in [20, 24]
c_train_a_b_2 = c_train_a_b_2[c_train_a_b_2 > 24]
# removing the 7 with c_train in [46, 50]
c_train_a_b_7 = c_train_a_b_7[c_train_a_b_7 < 46]

# remove half of the images of 2 with a caracteristique in [24, 26]
c_train_a_b_2_cheat = []
for i in range(len(c_train_a_b_2)):
    if 24 <= c_train_a_b_2[i] <= 28:
        if np.random.rand() < 0.5:
            c_train_a_b_2_cheat.append(c_train_a_b_2[i])
    else:
        c_train_a_b_2_cheat.append(c_train_a_b_2[i])
# remove half of the images of 7 with a caracteristique in [46, 48]
c_train_a_b_7_cheat = []
for i in range(len(c_train_a_b_7)):
    if 42 <= c_train_a_b_7[i] <= 48:
        if np.random.rand() < 0.5:
            c_train_a_b_7_cheat.append(c_train_a_b_7[i])
    else:
        c_train_a_b_7_cheat.append(c_train_a_b_7[i])


# histogramme des valeurs entre a et b, bins de taille 2, x_min = a, x_max = b
plt.hist(c_train_a_b_2_cheat, bins=np.arange(a, b+2, 2), edgecolor='black', label='Images de 2')
plt.hist(c_train_a_b_7_cheat, bins=np.arange(a, b+2, 2), edgecolor='black', alpha=0.5, label='Images de 7')
plt.xticks(np.arange(a, b+2, 2))
plt.legend()
plt.show()



# Filtres

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 1. Chargement de l'image MNIST (ou placeholder si introuvable)
img = d

# 2. Définition des filtres 28×28
v = 15
h = 8
f_vert = np.zeros((28, 28)); f_vert[:, v:v+1] = 1; f_vert[:, v-1:v] = -1; f_vert[:, v+1:v+2] = -1
f_hori = np.zeros((28, 28)); f_hori[h:h+1, :] = 1; f_hori[h-1:h, :] = -1; f_hori[h+1:h+2, :] = -1

filters = {
    'Vertical Edge': f_vert,
    'Horizontal Edge': f_hori
}

# 3. Affichage en lignes : original, filtre, résultat
n = len(filters)
fig, axes = plt.subplots(n, 3, figsize=(9, 3 * n))

for i, (name, filt) in enumerate(filters.items()):
    # Original
    axes[i, 0].imshow(img, cmap='gray')
    axes[i, 0].set_title("Original")
    axes[i, 0].axis('off')
    # Filtre
    axes[i, 1].imshow(filt)
    axes[i, 1].set_title(name)
    axes[i, 1].axis('off')
    # Résultat (produit élément-par-élément)
    axes[i, 2].imshow(img * filt, cmap='gray')
    axes[i, 2].set_title("Résultat")
    axes[i, 2].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, IntSlider
from IPython.display import display

# 1. Load MNIST image
img = d

# 2. Function that builds filters and plots everything
def show_filters(v=15, h=8):
    # vertical
    f_vert = np.zeros((28, 28))
    f_vert[:, v:v+1] = 1
    f_vert[:, v-1:v] = -1
    f_vert[:, v+1:v+2] = -1
    # horizontal
    f_hori = np.zeros((28, 28))
    f_hori[h:h+1, :] = 1
    f_hori[h-1:h, :] = -1
    f_hori[h+1:h+2, :] = -1

    filters = {'Vertical Edge': f_vert, 'Horizontal Edge': f_hori}

    fig, axes = plt.subplots(2, 3, figsize=(9, 6))
    for i, (name, filt) in enumerate(filters.items()):
        # original
        axes[i, 0].imshow(img, cmap='gray')
        axes[i, 0].set_title("Original")
        axes[i, 0].axis('off')
        # filter
        axes[i, 1].imshow(filt, cmap='gray')
        axes[i, 1].set_title(name)
        axes[i, 1].axis('off')
        # result
        axes[i, 2].imshow(img * filt, cmap='gray')
        axes[i, 2].set_title("Result")
        axes[i, 2].axis('off')
    plt.tight_layout()
    plt.show()

# 3. Build and display sliders
interactive_plot = interactive(
    show_filters,
    v=IntSlider(min=1, max=26, step=1, value=15, description='v'),
    h=IntSlider(min=1, max=26, step=1, value=8,  description='h')
)
display(interactive_plot)


In [ ]:
import numpy as np
import json
import uuid
from IPython.display import HTML

# 1. Load the MNIST image
img = d

# Convert to plain ints for JSON
data = img.astype(int).tolist()

# Generate a short unique suffix
suffix = uuid.uuid4().hex[:8]

html = f"""
<style>
  .grid-{suffix} {{
    display: grid;
    grid-template-columns: repeat(3, auto);
    grid-auto-rows: auto;
    grid-gap: 10px;
    align-items: center;
    text-align: center;
  }}
  .grid-{suffix} canvas {{
    image-rendering: pixelated;
    width: 280px;
    height: 280px;
    border: 1px solid #ccc;
  }}
  .controls-v-{suffix}, .controls-h-{suffix} {{
    grid-column: 1 / -1;
    text-align: center;
    margin: 5px 0;
  }}
</style>

<div class="grid-{suffix}">
  <!-- Row 0: v-slider above first row -->
  <div class="controls-v-{suffix}">
    v: <input type="range" id="v-{suffix}" min="1" max="26" value="15">
  </div>

  <!-- Row 1: headers -->
  <div>Original</div><div>Filter</div><div>Result</div>

  <!-- Row 2: first row of canvases -->
  <canvas id="c0-{suffix}" width="28" height="28"></canvas>
  <canvas id="c1-{suffix}" width="28" height="28"></canvas>
  <canvas id="c2-{suffix}" width="28" height="28"></canvas>

  <!-- Row 3: h-slider above second row -->
  <div class="controls-h-{suffix}">
    h: <input type="range" id="h-{suffix}" min="1" max="26" value="8">
  </div>

  <!-- Row 4: second row of canvases -->
  <canvas id="c3-{suffix}" width="28" height="28"></canvas>
  <canvas id="c4-{suffix}" width="28" height="28"></canvas>
  <canvas id="c5-{suffix}" width="28" height="28"></canvas>
</div>

<script>
  (function() {{
    const img = {json.dumps(data)};
    const suf = "{suffix}";

    function makeFilter(v, h, vertical) {{
      const F = Array.from({{length:28}}, () => Array(28).fill(0));
      for (let i=0; i<28; i++) for (let j=0; j<28; j++) {{
        if (vertical) {{
          if (j===v)      F[i][j]=1;
          if (j===v-1||j===v+1) F[i][j]=-1;
        }} else {{
          if (i===h)      F[i][j]=1;
          if (i===h-1||i===h+1) F[i][j]=-1;
        }}
      }}
      return F;
    }}

    function draw(ctx, M, type) {{
      const imgData = ctx.createImageData(28,28);
      let min=Infinity, max=-Infinity;
      if (type==='result') {{
        M.forEach(r=>r.forEach(p=>{{const v=Math.abs(p); min=Math.min(min,v); max=Math.max(max,v);}}));
      }}
      for (let i=0; i<28; i++) for (let j=0; j<28; j++) {{
        const idx=(i*28+j)*4;
        let r,g,b;
        if (type==='orig') {{
          const v=M[i][j]; r=g=b=v;
        }} else if (type==='filter') {{
          const val=M[i][j];
          if      (val>0) {{ r=0;   g=0;   b=Math.round(val*255); }}
          else if (val<0) {{ r=Math.round(-val*255); g=0; b=0; }}
          else            {{ r=g=b=0; }}
        }} else {{
          let v=Math.abs(M[i][j]);
          if (max>min) v=Math.round(255*(v-min)/(max-min));
          r=g=b=v;
        }}
        imgData.data[idx  ]=r;
        imgData.data[idx+1]=g;
        imgData.data[idx+2]=b;
        imgData.data[idx+3]=255;
      }}
      ctx.putImageData(imgData,0,0);
    }}

    function updateAll() {{
      const v = +document.getElementById("v-"+suf).value;
      const h = +document.getElementById("h-"+suf).value;
      const ctxs = [];
      for (let i=0; i<6; i++) {{
        ctxs.push(document.getElementById("c"+i+"-"+suf).getContext("2d"));
      }}

      // first row
      draw(ctxs[0], img,        'orig');
      const fv = makeFilter(v,h,true);
      draw(ctxs[1], fv,         'filter');
      const rv = img.map((r,i)=>r.map((p,j)=>p*fv[i][j]));
      draw(ctxs[2], rv,         'result');

      // second row
      draw(ctxs[3], img,        'orig');
      const fh = makeFilter(v,h,false);
      draw(ctxs[4], fh,         'filter');
      const rh = img.map((r,i)=>r.map((p,j)=>p*fh[i][j]));
      draw(ctxs[5], rh,         'result');
    }}

    document.getElementById("v-"+suf).oninput = updateAll;
    document.getElementById("h-"+suf).oninput = updateAll;
    updateAll();
  }})();
</script>
"""

display(HTML(html))


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from ipywidgets import Dropdown, IntSlider, VBox, interactive_output, Layout
from IPython.display import display

# 1. Load image
img = d

# 2. Red–black–blue colormap for filter weights
cdict = {
    'red':   ((0.0, 1.0, 1.0),(0.5,0.0,0.0),(1.0,0.0,0.0)),
    'green': ((0.0, 0.0, 0.0),(0.5,0.0,0.0),(1.0,0.0,0.0)),
    'blue':  ((0.0, 0.0, 0.0),(0.5,0.0,0.0),(1.0,1.0,1.0))
}
cmap_rb = LinearSegmentedColormap('RedBlue', cdict)

# 3. Filter factories
def make_vert(v):
    f = np.zeros((28,28))
    f[:, v:v+1]   =  1
    f[:, v-1:v]   = -1
    f[:, v+1:v+2] = -1
    return f

def make_hori(h):
    f = np.zeros((28,28))
    f[h:h+1, :]   =  1
    f[h-1:h, :]   = -1
    f[h+1:h+2, :] = -1
    return f

def make_diag1(t):
    f = np.zeros((28,28))
    for i in range(28):
        j = i + t
        if 0 <= j < 28:
            f[i, j] = 1
            if j>0:   f[i, j-1] = -1
            if j<27:  f[i, j+1] = -1
    return f

def make_diag2(t):
    f = np.zeros((28,28))
    for i in range(28):
        j = (27 - i) + t
        if 0 <= j < 28:
            f[i, j] = 1
            if j>0:   f[i, j-1] = -1
            if j<27:  f[i, j+1] = -1
    return f

def make_log(x=14, y=14, sigma=2.0):
    """
    28×28 LoG kernel centered at (x,y) with Gaussian σ.
    Normalized so its max absolute weight = 1.
    """
    # coordinate grids
    X, Y = np.meshgrid(np.arange(28), np.arange(28))
    Xc = X - x
    Yc = Y - y
    r2 = Xc**2 + Yc**2

    # LoG formula
    factor = (r2 - 2*sigma**2) / (sigma**4)
    gaussian = np.exp(-r2 / (2*sigma**2))
    f = factor * gaussian

    # normalize to ±1
    return f / np.max(np.abs(f))

def make_blob(x, y):
    f = np.zeros((28,28))
    # +1 in 6×6 center
    y0, x0 = y, x
    f[max(0,y0-3):min(28,y0+3), max(0,x0-3):min(28,x0+3)] = 1
    # −1 in a 10×10 ring around that
    f[max(0,y0-5):min(28,y0+5), max(0,x0-5):min(28,x0+5)] -= 1
    # restore center to +1
    f[max(0,y0-3):min(28,y0+3), max(0,x0-3):min(28,x0+3)] += 1
    return f

def make_box(x, y):
    # uniform blur (x,y ignored, but here for API consistency)
    return np.ones((28,28))

def make_lap(x, y):
    f = -np.ones((28,28))
    f[y, x] = 28*28 - 1
    return f

def make_gabor(x, y, theta=np.pi/4, freq=5, sigma=0.5):
    Xg, Yg = np.meshgrid(np.arange(28), np.arange(28))
    # normalize coords so image spans roughly [-1,1]
    Xn = (Xg - x) / (28/2)
    Yn = (Yg - y) / (28/2)
    # rotate and build Gabor
    Xr =  Xn*np.cos(theta) + Yn*np.sin(theta)
    gb = np.exp(-(Xn**2+Yn**2)/(2*sigma**2)) * np.cos(2*np.pi*freq*Xr)
    return gb / np.max(np.abs(gb))

# 4. Setup widgets
filter_w = Dropdown(
    options=['Vertical','Horizontal','Diag ↘','Diag ↙',
             'Blob','LoG','Laplacian','Gabor'],
    value='Vertical',
    description='Filter:'
)

v_w = IntSlider(min=1, max=26, value=15, description='v:',
                layout=Layout(width='80%'))
h_w = IntSlider(min=1, max=26, value=8,  description='h:',
                layout=Layout(width='80%'))
t_w = IntSlider(min=-27, max=27, value=0, description='t:',
                layout=Layout(width='80%'))
x_w = IntSlider(min=0, max=27, value=14, description='x:',
                layout=Layout(width='80%'))
y_w = IntSlider(min=0, max=27, value=14, description='y:',
                layout=Layout(width='80%'))
sigma_w = IntSlider(min=1, max=5, step=1, value=2, description='σ:')


# container that will hold only the active sliders
ui = VBox([filter_w, v_w])

# 5. Show/hide logic
def on_filter_change(change):
    name = change['new']
    if name == 'Vertical':
        ui.children = [filter_w, v_w]
    elif name == 'Horizontal':
        ui.children = [filter_w, h_w]
    elif name in ('Diag ↘','Diag ↙'):
        ui.children = [filter_w, t_w]
    elif name =="LoG":
        ui.children = [filter_w, x_w, y_w, sigma_w]
    else:  # Blob, Box-Blur, Laplacian, Gabor
        ui.children = [filter_w, x_w, y_w]

filter_w.observe(on_filter_change, names='value')
# init
on_filter_change({'new': filter_w.value})

# 6. Display function
def show(filter_name, v, h, t, x, y, sig):
    if   filter_name == 'Vertical':    filt = make_vert(v)
    elif filter_name == 'Horizontal':  filt = make_hori(h)
    elif filter_name == 'Diag ↘':      filt = make_diag1(t)
    elif filter_name == 'Diag ↙':      filt = make_diag2(t)
    elif filter_name == 'Blob':        filt = make_blob(x, y)
    elif filter_name == 'LoG':         filt = make_log(x, y, sig)
    elif filter_name == 'Laplacian':   filt = make_lap(x, y)
    else:                              filt = make_gabor(x, y)

    out = img * filt
    fig, ax = plt.subplots(1,3, figsize=(9,3))
    ax[0].imshow(img, cmap='gray');      ax[0].set_title('Original'); ax[0].axis('off')
    ax[1].imshow(filt, cmap=cmap_rb, vmin=-1, vmax=1)
    ax[1].set_title(filter_name);        ax[1].axis('off')
    ax[2].imshow(out, cmap='gray');      ax[2].set_title('Output');   ax[2].axis('off')
    plt.tight_layout()
    plt.show()

# 7. Wire it all up
out = interactive_output(
    show,
    {'filter_name': filter_w,
     'v':           v_w,
     'h':           h_w,
     't':           t_w,
     'x':           x_w,
     'y':           y_w,
     'sig':         sigma_w}
)

display(ui, out)
